In [44]:
import numpy as np
import pandas as pd
import json
from utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn import preprocessing

In [45]:
truth_df = pd.DataFrame(columns=['id','truthMedian','truthClass','truthMean'])
with open('data/truth.jsonl') as data:
    for labelobj in data:
        truth = json.loads(labelobj)
        truthlabel = {'id': truth['id'], 'truthMedian': truth['truthMedian'], 'truthClass': truth['truthClass'], 'truthMean': truth['truthMean']}
        truth_df = truth_df.append(truthlabel, ignore_index = True)
truth_df.head()   

,id,truthMedian,truthClass,truthMean
0,858464162594172928,1.000000,clickbait,1.000000
1,858462320779026433,0.000000,no-clickbait,0.133333
2,858460992073863168,0.333333,no-clickbait,0.400000
3,858459539296980995,0.333333,no-clickbait,0.266667
4,858455355948384257,0.000000,no-clickbait,0.000000


In [46]:
instances_df = pd.DataFrame(columns=['id','postText'])
with open('data/instances.jsonl') as data:
	for instanceobj in data:
		instance = json.loads(instanceobj)
		instancerow = {'id': instance['id'], 'postText': instance['postText']}
		instances_df = instances_df.append(instancerow, ignore_index=True)
instances_df.head()

,id,postText
0,858462320779026433,[UK’s response to modern slavery leaving victi...
1,858421020331560960,[this is good]
2,858368123753435136,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,858323428260139008,[Meet the happiest #dog in the world!]
4,858283602626347008,[Tokyo's subway is shut down amid fears over a...


In [47]:
dataset = instances_df.join(truth_df.set_index('id'), on='id')
dataset = dataset.drop(labels='id',axis=1)
for i in range(len(dataset)):
    dataset['postText'].values[i] = dataset['postText'].values[i][0]
dataset['postText'].dropna(inplace=True)
dataset.head()

,postText,truthMedian,truthClass,truthMean
0,UK’s response to modern slavery leaving victim...,0.000000,no-clickbait,0.133333
1,this is good,1.000000,clickbait,1.000000
2,"The ""forgotten"" Trump roast: Relive his brutal...",0.333333,no-clickbait,0.466667
3,Meet the happiest #dog in the world!,1.000000,clickbait,0.933333
4,Tokyo's subway is shut down amid fears over an...,0.000000,no-clickbait,0.000000


In [48]:
# Convert the 'no-clickbait' or 'clickbait' to binary indicator
dataset['truthClass'] = dataset['truthClass'].apply(classToBinary)
# Convert floating number in 'truthMedian' column to integer
dataset['truthMedian'] = dataset['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
dataset['postText'] = dataset['postText'].apply(cleanText)
dataset.head()

,postText,truthMedian,truthClass,truthMean
0,uks response to modern slavery leaving victims...,0,0,0.133333
1,this is good,3,1,1.000000
2,the forgotten trump roast relive his brutal 20...,1,0,0.466667
3,meet the happiest dog in the world,3,1,0.933333
4,tokyos subway is shut down amid fears over an ...,0,0,0.000000


In [49]:
# import the glove word embedding file
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.100d.txt')

In [50]:
# length of titles with longest words
maxLen = maxLengthInPostText(dataset)
maxLen

25

In [51]:
Indices = sentences_to_indices(X_train,word_to_index, maxLen)
print("X_Train_indices =\n", Indices.shape)

X_Train_indices =
 (17584, 25)


In [66]:
def ClickBait_BiLSTM(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)
    # dropout
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(128, return_sequences=False))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 2 units
    X = Dense(4)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)  
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X) 
    return model

In [67]:
BiLSTM_model = ClickBait_BiLSTM((maxLen,), word_to_vec_map, word_to_index)
BiLSTM_model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 25, 100)           40000100  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 25, 256)           234496    
_________________________________________________________________
dropout_9 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 25, 256)           394240    
_________________________________________________________________
dropout_10 (Dropout)         (None, 25, 256)           0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 256)              

In [68]:
BiLSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 4)
X_train_indices.shape

(17584, 25)

In [70]:
BiLSTM_model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 32, shuffle=True)

Epoch 1/10
550/550 [==============================] - 32s 58ms/step - loss: 1.0080 - accuracy: 0.5514
Epoch 2/10
550/550 [==============================] - 33s 60ms/step - loss: 0.9465 - accuracy: 0.5780
Epoch 3/10
550/550 [==============================] - 33s 60ms/step - loss: 0.9236 - accuracy: 0.5875
Epoch 4/10
550/550 [==============================] - 33s 61ms/step - loss: 0.9012 - accuracy: 0.59480s - loss: 0.9005 
Epoch 5/10
550/550 [==============================] - 33s 60ms/step - loss: 0.8797 - accuracy: 0.6082
Epoch 6/10
550/550 [==============================] - 25s 46ms/step - loss: 0.8545 - accuracy: 0.6164
Epoch 7/10
550/550 [==============================] - 23s 42ms/step - loss: 0.8313 - accuracy: 0.6263
Epoch 8/10
550/550 [==============================] - 31s 56ms/step - loss: 0.7851 - accuracy: 0.6488
Epoch 9/10
550/550 [==============================] - 36s 66ms/step - loss: 0.7444 - accuracy: 0.6720
Epoch 10/10
550/550 [==============================] - 37s 66ms/

In [71]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error, classification_report
y_train_pred_oh = BiLSTM_model.predict(X_train_indices)
y_train_pred_binary = onehot_to_binary(y_train_pred_oh)
Y_train_binary = onehot_to_binary(Y_train_oh)

print("LSTM Training Accuracy")
print('Accuracy %s' % accuracy_score(Y_train_binary, y_train_pred_binary))
print('Precision %s' % precision_score(Y_train_binary, y_train_pred_binary))
print('Recall %s' % recall_score(Y_train_binary, y_train_pred_binary))
print('F1 score: %s' % f1_score(Y_train_binary, y_train_pred_binary))
print('MSE %s' % mean_squared_error(Y_train_mean, y_train_pred_binary))

LSTM Training Accuracy
Accuracy 0.9059940855323021
Precision 0.8357034027425089
Recall 0.7658831743076565
F1 score: 0.7992714025500911
MSE 0.09458902031760782


In [72]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
y_pred_onehot = BiLSTM_model.predict(X_test_indices)
y_pred_binary = onehot_to_binary(y_pred_onehot)

print("LSTM Testing Accuracy")
print('Accuracy %s' % accuracy_score(Y_test, y_pred_binary))
print('Precision %s' % precision_score(Y_test, y_pred_binary))
print('Recall %s' % recall_score(Y_test, y_pred_binary))
print('F1 score: %s' % f1_score(Y_test, y_pred_binary))
print('MSE %s' % mean_squared_error(Y_test_mean, y_pred_binary))

LSTM Testing Accuracy
Accuracy 0.8372569089048106
Precision 0.6763285024154589
Recall 0.603448275862069
F1 score: 0.6378132118451026
MSE 0.11700216080538066


In [73]:
print("LSTM Testing report")
print(classification_report(Y_test, y_pred_binary))

LSTM Testing report
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      1490
           1       0.68      0.60      0.64       464

    accuracy                           0.84      1954
   macro avg       0.78      0.76      0.77      1954
weighted avg       0.83      0.84      0.83      1954



In [74]:
print('Minimum MSE %s' % mean_squared_error(Y_test_mean, Y_test))

Minimum MSE 0.07735698851201318


In [75]:
# Save our model
BiLSTM_model.save("BiLSTM_model.h5")

In [76]:
# Error Analysis
for i in range(100):
    if Y_test[i] - y_pred_binary[i] != 0:
        print(X_test[i])
        print("Actual Label",Y_test[i])
        print("Prediction Lable",y_pred_binary[i])
        print("Prediction",y_pred_onehot[i])
        print("-------------")

death is part of working on the mountain
Actual Label 1
Prediction Lable 0
Prediction [0.61711687 0.34225258 0.03849593 0.00213457]
-------------
should we be allowed to use superannuation on house deposits
Actual Label 1
Prediction Lable 0
Prediction [0.85207176 0.13587472 0.01117451 0.0008791 ]
-------------
chinese companies success in reducing debt isnt all it seems  via bfly
Actual Label 1
Prediction Lable 0
Prediction [0.1615168  0.5215397  0.28216648 0.03477708]
-------------
how the ball brothers came to rule in basketball
Actual Label 1
Prediction Lable 0
Prediction [0.13078845 0.45198175 0.3577132  0.05951652]
-------------
europe could be heading for its first far right leader since wwii
Actual Label 1
Prediction Lable 0
Prediction [0.67337555 0.29133207 0.03332807 0.00196432]
-------------
ges jeff immelt on trump and globalization
Actual Label 1
Prediction Lable 0
Prediction [9.4157064e-01 5.8237504e-02 1.9021989e-04 1.5401434e-06]
-------------
said in their resignation l

In [77]:
# Utilize our model and test the real-world headlines
def test(headline):
    headline = cleanText(headline)
    headline_np = np.array([headline])
    indices = sentences_to_indices(headline_np, word_to_index, max_len = maxLen)
    #y_pred_onehot = lstm_model.predict(test_indices)
    y_pred_onehot = BiLSTM_model.predict(indices)
    y_pred_binary = onehot_to_binary(y_pred_onehot)
    return True if y_pred_binary == [1] else False

In [78]:
headline = "Here are 10 things you may not know"
test(headline)

True